In [1]:
from pyBKT.models import Model, Roster, StateType
import numpy as np

In [ ]:

# Paso 1: Entrenamos el modelo en una habilidad concreta
model = Model(seed=42)
model.fit(data_path='data/as_red.csv', multiprior="user_id", forgets = True)
# # Paso 2: Creamos el Roster con estudiantes y la habilidad que se va a seguir
# roster = Roster(students=['78564'], skills='Volume Cylinder', model=model)

# # Paso 3: Estado inicial de Olmo (antes de responder nada)
# print("Estado inicial:")
# print(f"¿Domina?: {roster.get_state_type('Volume Cylinder', '78564')}")
# print(f"P(dominio): {roster.get_mastery_prob('Volume Cylinder', '78564'):.4f}")
# print(f"P(respuesta correcta): {roster.get_correct_probs('Volume Cylinder')['78564']:.4f}")
# print()
# model_a = roster.get_model()
# # Paso 4: Secuencia simulada de respuestas (1 = acierto, 0 = fallo)
# respuestas = [1, 0, 1, 1, 1]  # Puedes cambiar esta lista

# print("Actualizando estado tras cada respuesta:")
# for i, respuesta in enumerate(respuestas, 1):
#     state = roster.update_state('Volume Cylinder', '78564', respuesta)
#     print(f"Paso {i} - Respuesta: {'✔️' if respuesta == 1 else '❌'}")
#     print(f"  P(dominio): {state.get_mastery_prob():.4f}")
#     print(f"  P(correcto): {state.get_correct_prob():.4f}")
#     print(f"  ¿Domina?: {roster.get_state_type('Volume Cylinder', '78564') == StateType.MASTERED}")
#     print()
# model_b = roster.get_model()
# # Paso 5: Comprobación final
# if roster.get_state_type('Volume Cylinder', '78564') == StateType.MASTERED:
#     print("🎉 ¡Olmo ha dominado la habilidad!")
# else:
#     print("🔄 Olmo aún no domina la habilidad.")
    
    
# roster_2 = Roster(students=['78564'], skills='Volume Cylinder', model=model_b)
# print("Estado inicial:")
# print(f"¿Domina?: {roster_2.get_state_type('Volume Cylinder', '78564')}")
# print(f"P(dominio): {roster_2.get_mastery_prob('Volume Cylinder', '78564'):.4f}")
# print(f"P(respuesta correcta): {roster_2.get_correct_probs('Volume Cylinder')['78564']:.4f}")
# print()

KeyError: 'Volume Cylinder'

In [240]:
import pandas as pd
from pyBKT.models import Model

# — 1) Entrena el modelo con multilearn sobre user_id y olvido —
#    de manera que aprende learns/forgets por alumno
m = Model(seed=42)
m.fit(
    data_path='data/as_red.csv',
    multiprior='user_id',
    forgets=True
)

In [223]:
import pandas as pd
from pyBKT.models import Model

# — supongamos ya tienes entrenado tu modelo ‘m’ con 
#   multilearn='user_id' y forgets=True —

# 1) Obtén el DataFrame de parámetros
params = m.params().reset_index()  
# columnas: ['skill','param','class','value']

# 2) Filtra sólo learns y forgets
learn_df  = params[params.param == 'learns' ].copy()
forget_df = params[params.param == 'forgets'].copy()

# 3) Pivot para que:
#    index = class (→ user_id), columns = skill, values = value
learn_pivot  = learn_df.pivot(index='class', columns='skill', values='value')
forget_pivot = forget_df.pivot(index='class', columns='skill', values='value')

# 4) Renombrar el índice a algo semántico
learn_pivot .index.name = 'user_id'
forget_pivot.index.name = 'user_id'

# 5) (Opcional) Añadir sufijos a las columnas
learn_pivot.columns  = [f"{skill}_learn"  for skill in learn_pivot.columns]
forget_pivot.columns = [f"{skill}_forget" for skill in forget_pivot.columns]

# 6) Unir ambos pivots en un solo DataFrame
results = pd.concat([learn_pivot, forget_pivot], axis=1).reset_index()

# 7) Guarda o muestra
results.to_csv('learns_forgets_per_user.csv', index=False)
print(results)


      user_id  Absolute Value_learn  Addition Whole Numbers_learn  \
0       52613                   NaN                           NaN   
1       53167                   NaN                           NaN   
2       64525                   NaN                           NaN   
3       64535               0.44793                       0.99999   
4       64634                   NaN                           NaN   
...       ...                   ...                           ...   
1364    96296                   NaN                           NaN   
1365    96297                   NaN                           NaN   
1366    96298                   NaN                           NaN   
1367    96299                   NaN                           NaN   
1368  Default               0.44394                       0.99999   

      Addition and Subtraction Fractions_learn  \
0                                          NaN   
1                                          NaN   
2                    

In [261]:
import pandas as pd

# --- 1) obtenemos el DataFrame completo de parámetros ---
print(m.params())
params = m.params().reset_index()
# columnas: ['skill','param','class','value']
print(params)
# --- 2) filtramos SOLO las filas donde class == 'default' ---
# Convertimos a minúsculas para evitar problemas de mayúsculas
params['class'] = params['class'].str.lower()
default_params = params[params['class'] == 'default']
# print(default_params)
# --- 3) pivot para tener:
#      index = skill, columns = param, values = value
table = default_params.pivot(index='skill', columns='param', values='value')

# --- 4) reordenar y renombrar las columnas a gusto ---
table = table[['prior', 'learns', 'guesses', 'slips', 'forgets']]



# Convertir a DF
table = table.reset_index()
# Renombrar el índice a algo semántico
# table.columns.name = 'skill'
# Quitar indices a la tabla
table.columns.name = None
print(table)


                                              value
skill                       param   class          
Distributive Property       prior   default 0.00000
                            learns  52613   1.00000
                                    Default 1.00000
                            guesses default 0.50000
                            slips   default 0.00000
...                                             ...
Finding Slope From Equation slips   default 0.33333
                            forgets 92246   0.00000
                                    92250   0.00000
                                    92251   0.00000
                                    Default 0.00000

[7345 rows x 1 columns]
                            skill    param    class   value
0           Distributive Property    prior  default 0.00000
1           Distributive Property   learns    52613 1.00000
2           Distributive Property   learns  Default 1.00000
3           Distributive Property  guesses  default 0.50000

In [217]:
results_per_user.loc[results_per_user["user_id"] == "78552"]


,user_id,Volume Cylinder_learn,Volume Cylinder_forget,Box and Whisker_learn,Box and Whisker_forget,Conversion of Fraction Decimals Percents_learn,Conversion of Fraction Decimals Percents_forget,Volume Rectangular Prism_learn,Volume Rectangular Prism_forget,Addition and Subtraction Integers_learn,...,Scale Factor_learn,Scale Factor_forget,Midpoint_learn,Midpoint_forget,Rotations_learn,Rotations_forget,D.4.8-understanding-concept-of-probabilities_learn,D.4.8-understanding-concept-of-probabilities_forget,Distributive Property_learn,Distributive Property_forget


In [147]:
model.params().loc["Distributive Property"]
# Guardar elementos
# Para cada skill, guardar learns de cada usuario, forgets de cada usuario, guesses y slips
model.params()


value
skill                       param   class          
Distributive Property       prior   default 0.00000
                            learns  52613   1.00000
                                    Default 1.00000
                            guesses default 0.50000
                            slips   default 0.00000
...                                             ...
Finding Slope From Equation slips   default 0.33333
                            forgets 92246   0.00000
                                    92250   0.00000
                                    92251   0.00000
                                    Default 0.00000

[7345 rows x 1 columns]

In [7]:
print(help(model))

Help on Model in module pyBKT.models.Model object:

class Model(builtins.object)
 |  Model(**kwargs)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, **kwargs)
 |      Constructs a BKT Model. Takes arguments parallel, num_fits, seed, defaults,
 |      and any model variant(s) that may be used. Note that all of these can be modified
 |      during fit/crossvalidation time.
 |      
 |      >>> model = Model(seed = 42)
 |      >>> model
 |      Model(parallel=True, num_fits=5, seed=42, defaults=None)
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  crossvalidate(self, data=None, data_path=None, metric=<function rmse at 0x7c63b31dbe60>, **kwargs)
 |      Crossvalidates (trains and evaluates) the BKT model. Takes the data, metric, and any
 |      arguments that would be passed to the fit function (skills, number of initialization fits, 
 |      default column names, parallelization, and model types) as arguments.
 |      
 |      >>> model = Model(seed = 42)
 |      >>> mo

In [262]:
import pandas as pd

# Mini–dataset simulado  (user_id, skill, correct, order_id, item_id)
df = pd.DataFrame({
    'user_id' : ['alice','alice','bob','bob']*3,
    'skill_name'   : ['fractions']*6 + ['equations']*6,
    'correct' : [0,1,0,0,  1,1,1,0,  0,1,1,1],
    'item_id' : ['A','B','A','B']*3          # para multigs / multilearn
})

df['order_id'] = df.groupby(['user_id','skill_name']).cumcount() + 1
print(df)

   user_id skill_name  correct item_id  order_id
0    alice  fractions        0       A         1
1    alice  fractions        1       B         2
2      bob  fractions        0       A         1
3      bob  fractions        0       B         2
4    alice  fractions        1       A         3
5    alice  fractions        1       B         4
6      bob  equations        1       A         1
7      bob  equations        0       B         2
8    alice  equations        0       A         1
9    alice  equations        1       B         2
10     bob  equations        1       A         3
11     bob  equations        1       B         4


In [8]:
import pandas as pd
from pyBKT.models import Model

# ─────────────────────────────────────────
# 1) Tus datos de ejemplo
# ─────────────────────────────────────────
df = pd.DataFrame({
    'user_id'   : ['alice','alice','bob','bob']*3,
    'skill_name': ['fractions']*6 + ['equations']*6,
    'correct'   : [0,1,0,0,  1,1,  1,0,0,1,1,1],
    'item_id'   : ['A','B','A','B']*3
})
df['order_id'] = df.groupby(['user_id','skill_name']).cumcount() + 1

# ─────────────────────────────────────────
# 2) Creamos el modelo y fijamos priors «a priori»
# ─────────────────────────────────────────
model = Model(seed=42)

# Supongamos que queremos que:
#   - fractions tenga prior = 0.3
#   - equations tenga prior = 0.7
model.coef_ = {
    # 'fractions' : {'prior': 0.3},
    # 'equations' : {'prior': 0.7}
}

# ─────────────────────────────────────────
# 3) Entrenamos «congelando» solo el prior
# ─────────────────────────────────────────
model.fit(
    data=df,
    multiprior='user_id',  # multiprior='user_id' o 'item_id'
    # skills=['fractions','equations'],
    # fixed=True,     # fija los parámetro indicados en model.coef_ (solo prior)
    forgets=True,   # opcional, si quieres estimar olvido
    defaults={
        'skill_name' : 'skill_name',
        'correct'    : 'correct',
        'order_id'   : 'order_id'
    }
)

# ─────────────────────────────────────────
# 4) Visualizamos parámetros finales
# ─────────────────────────────────────────
print(model.params())


                            value
skill     param   class          
fractions prior   default 0.00000
          learns  Default 0.48813
                  alice   0.14653
                  bob     0.01945
          guesses default 0.02845
          slips   default 0.08062
          forgets Default 0.00656
                  alice   0.00000
                  bob     0.00000
equations prior   default 0.00000
          learns  Default 0.63193
                  alice   0.18204
                  bob     0.73968
          guesses default 0.43978
          slips   default 0.10387
          forgets Default 0.64113
                  alice   0.00000
                  bob     0.00000


In [285]:
import pandas as pd

# --- 1) obtenemos el DataFrame completo de parámetros ---
params = model.params().reset_index()
# columnas: ['skill','param','class','value']
# --- 2) filtramos SOLO las filas donde class == 'default' ---
# Convertimos a minúsculas para evitar problemas de mayúsculas
params['class'] = params['class'].str.lower()
default_params = params[params['class'] == 'default']
# print(default_params)
# --- 3) pivot para tener:
#      index = skill, columns = param, values = value
table = default_params.pivot(index='skill', columns='param', values='value')

# --- 4) reordenar y renombrar las columnas a gusto ---
table = table[['prior', 'learns', 'guesses', 'slips', 'forgets']]



# Convertir a DF
table = table.reset_index()
# Renombrar el índice a algo semántico
# table.columns.name = 'skill'
# Quitar indices a la tabla
table.columns.name = None
print(table)


       skill   prior  learns  guesses   slips  forgets
0  equations 0.00000 0.63193  0.43978 0.10387  0.64113
1  fractions 0.00000 0.48813  0.02845 0.08062  0.00656


In [286]:
params = model.params().reset_index()  

# Filtra sólo learns y forgets
learn_df  = params[params.param == 'learns' ].copy()
forget_df = params[params.param == 'forgets'].copy()

# Pivot para que:
#    index = class (→ user_id), columns = skill, values = value
learn_pivot  = learn_df.pivot(index='class', columns='skill', values='value')
forget_pivot = forget_df.pivot(index='class', columns='skill', values='value')

learn_pivot .index.name = 'user_id'
forget_pivot.index.name = 'user_id'

# Columnas para aprender y olvidar
learn_pivot.columns  = [f"{skill}_learn"  for skill in learn_pivot.columns]
forget_pivot.columns = [f"{skill}_forget" for skill in forget_pivot.columns]

# Unimos en un solo dataframe
results = pd.concat([learn_pivot, forget_pivot], axis=1).reset_index()
results = results[results['user_id'] != 'Default']

results

,user_id,equations_learn,fractions_learn,equations_forget,fractions_forget
1,alice,0.18204,0.14653,0.00000,0.00000
2,bob,0.73968,0.01945,0.00000,0.00000


In [9]:
from pyBKT.models import Model

# --- 1. BKT base (referencia) ---
base = Model(seed=0)
base.fit(data=df)
print("BASE", base.params())
print("\n")

# --- 2. KT-IDEM: guess/slip por ítem ---
idem = Model(seed=0)
idem.fit(data=df,
         multigs='item_id',
         )               # columna que define la clase de ítem
        #  defaults={'multigs':'skill_name'})
print("KT-IDEM", idem.params())
print("\n")

# --- 3. KT-PPS: prior por estudiante ---
pps = Model(seed=0)
pps.fit(data=df, skills='fractions',
        multiprior='user_id',
        defaults={'multiprior':'user_id'})
print("KT-PPS", pps.params().loc['fractions'])   # solo muestra 'prior default', pero internamente hay uno por user_id
print("\n")

# --- 4. BKT+Forget (olvido global) ---
forget = Model(seed=0)
forget.fit(data=df, skills='fractions', forgets=True)
print("BKT+Forget", forget.params().loc['fractions'])
print("\n")

# --- 5. Item Order Effect ---
# Necesita una columna que represente el par anterior-actual (problema_id)
df['problem_id'] = df['item_id']          # aquí lo reutilizamos
pair = Model(seed=0)
pair.fit(data=df, skills='fractions',
         multipair='problem_id',
         defaults={'multipair':'problem_id'})
print("Item-Order", pair.params().loc['fractions'])
print("\n")

# --- 6. Item Learning Effect: learn rate por ítem ---
ilearn = Model(seed=0)
ilearn.fit(data=df, skills='fractions',
           multilearn='item_id',
           defaults={'multilearn':'item_id'})
print("Item-Learning", ilearn.params().loc['fractions'])
print("\n")


BASE                             value
skill     param   class          
fractions prior   default 0.00268
          learns  default 0.53239
          guesses default 0.00040
          slips   default 0.02385
          forgets default 0.00000
equations prior   default 0.48203
          learns  default 0.55455
          guesses default 0.46401
          slips   default 0.25631
          forgets default 0.00000


KT-IDEM                             value
skill     param   class          
fractions prior   default 0.00084
          learns  default 0.51526
          guesses A       0.00002
                  B       0.00634
          slips   A       0.00167
                  B       0.01702
          forgets default 0.00000
equations prior   default 0.00416
          learns  default 0.50713
          guesses A       0.55272
                  B       0.27937
          slips   A       0.00200
                  B       0.02359
          forgets default 0.00000


KT-PPS                   value


In [22]:
params = pps.params().reset_index()
print(params)


       skill    param    class   value
0  fractions    prior  default 0.00000
1  fractions   learns  Default 0.49514
2  fractions   learns    alice 0.11594
3  fractions   learns      bob 0.00769
4  fractions  guesses  default 0.00860
5  fractions    slips  default 0.05955
6  fractions  forgets  Default 0.00000
7  fractions  forgets    alice 0.00000
8  fractions  forgets      bob 0.00000


In [33]:
skill = ["fractions"]
user_id = "alice"
value = "learns"
params.loc[(params["class"] == user_id) & (params["param"] == "learns")]["value"].values[0]

0.1159436510500644

In [7]:
dummy = pd.DataFrame({
    'user_id'   : ['Ana'],
    'skill_name': ['fractions'],
    'correct'   : [-1],            # no respuesta
    'order_id'  : [1]
})

prior_carlos = m.predict(data=dummy).state_predictions.iloc[0]
print("Prior de Ana =", prior_carlos)
m.predict(data=df[df.user_id=='Ana'])

NameError: name 'm' is not defined

In [36]:
roster = Roster(['alice','bob'],'fractions', model=pps)

# fijar manualmente los priors aprendidos
ana_prior    = pps.predict(data=df[df.user_id=='alice']).state_predictions.iloc[0]
pedro_prior  = pps.predict(data=df[df.user_id=='bob']).state_predictions.iloc[0]

roster.skill_rosters['fractions'].students['alice'].current_state['state_prediction']   = ana_prior
roster.skill_rosters['fractions'].students['bob'].current_state['state_prediction'] = pedro_prior

# Comprobación de que los priors son correctos
print("Ana", roster.skill_rosters['fractions'].students['alice'].current_state['state_prediction'])
print("Pedro", roster.skill_rosters['fractions'].students['bob'].current_state['state_prediction'])

for _ in range(3):
    roster.update_state('fractions','Ana',1)
    roster.update_state('fractions','Pedro',1)
# Ana falla 1 vez

# …pasan 2 interacciones SIN pregunta (-1) ⇒ solo afecta a rosters
for _ in range(2):
    roster.update_state('fractions','Ana',-1)  # -1 = no respuesta
    roster.update_state('fractions','Pedro',0)  # -1 = no respuesta

print("P(L) sin olvido :", roster.get_mastery_prob('fractions','Ana'))
print("P(L) sin olvido :", roster.get_mastery_prob('fractions','Pedro'))

Ana 0.1159436510500644
Pedro 0.0076944665875581285


ValueError: student name not found in roster for this skill

In [105]:
forget = Model(seed=0)
forget.fit(data_path='data/as_red.csv', multiprior = "user_id", forgets=True)
print("BKT+Forget", forget.params())
print("\n")
# Guardar modelo
forget.save('forget_model.pkl')
# Cargar modelo
forget = Model()
forget.load(loc = 'forget_model.pkl')
# Comprobar que el modelo cargado es el mismo
print("Modelo cargado:", forget.params())

BKT+Forget                                               value
skill                       param   class          
Distributive Property       prior   default 0.00000
                            learns  52613   1.00000
                                    Default 1.00000
                            guesses default 0.50000
                            slips   default 0.00000
...                                             ...
Finding Slope From Equation slips   default 0.33333
                            forgets 92246   0.00000
                                    92250   0.00000
                                    92251   0.00000
                                    Default 0.00000

[7345 rows x 1 columns]


Modelo cargado:                                               value
skill                       param   class          
Distributive Property       prior   default 0.00000
                            learns  52613   1.00000
                                    Default 1.00000
          

In [70]:
import numpy as np
from pyBKT.models import Roster, StateType

test = pd.DataFrame({
    'user_id':['Ana']*6+['Pedro']*6,
    'skill_name':['fractions']*12,
    'correct':[1,1,1,1,1,1,0,0,1,0,1,1],
    'order_id':range(1,13)
})
# entrenamos dos modelos con el mismo learn pero…
base  = Model(); base.fit(data=test, skills='fractions', multiprior='user_id')
fog   = Model(); fog.fit(data=test, skills='fractions', multiprior='user_id', forgets=True)

# seguimiento
roster_base = Roster(['Ana',"Pedro"],'fractions',model=base)
roster_fog  = Roster(['Ana', "Pedro"],'fractions',model=fog)

# fijar manualmente los priors aprendidos
ana_prior    = m.predict(data=df[df.user_id=='Ana']).state_predictions.iloc[0]
pedro_prior  = m.predict(data=df[df.user_id=='Pedro']).state_predictions.iloc[0]

roster_fog.skill_rosters['fractions'].students['Ana'].current_state['state_prediction']   = ana_prior
roster_fog.skill_rosters['fractions'].students['Pedro'].current_state['state_prediction'] = pedro_prior
# Ana acierta 3 veces
for _ in range(3):
    roster_base.update_state('fractions','Ana',1)
    roster_fog.update_state('fractions','Ana',1)
    roster_base.update_state('fractions','Pedro',1)
    roster_fog.update_state('fractions','Pedro',1)
# Ana falla 1 vez

# …pasan 2 interacciones SIN pregunta (-1) ⇒ solo afecta a rosters
for _ in range(2):
    roster_base.update_state('fractions','Ana',-1)  # -1 = no respuesta
    roster_fog.update_state('fractions','Ana',-1)
    roster_base.update_state('fractions','Pedro',-1)  # -1 = no respuesta
    roster_fog.update_state('fractions','Pedro',-1)

print("P(L) sin olvido :", roster_base.get_mastery_prob('fractions','Ana'))
print("P(L) con olvido:", roster_fog .get_mastery_prob('fractions','Ana'))
print("P(L) sin olvido :", roster_base.get_mastery_prob('fractions','Pedro'))
print("P(L) con olvido:", roster_fog .get_mastery_prob('fractions','Pedro'))

P(L) sin olvido : 0.9612676986720807
P(L) con olvido: 0.9612778085186247
P(L) sin olvido : 0.9612676986720807
P(L) con olvido: 0.9140208205645991


In [54]:
import pandas as pd

# Tres estudiantes, todos sobre la misma habilidad "fractions"
test = pd.DataFrame({
    'user_id'   : ['Ana']*6 + ['Bob']*4 + ['Carlos']*3,
    'skill_name': ['fractions']*13,
    'correct'   : [1,1,1,1,1,1,            # Ana acierta siempre
                   0,0,1,1,                # Bob empieza fallando
                   1,0,1],                 # Carlos alterna
    # el order_id debe reiniciarse por (user_id, skill)
})

# Generamos order_id correcto por alumno-skill
test['order_id'] = test.groupby(['user_id','skill_name']).cumcount() + 1

print(test)


   user_id skill_name  correct  order_id
0      Ana  fractions        1         1
1      Ana  fractions        1         2
2      Ana  fractions        1         3
3      Ana  fractions        1         4
4      Ana  fractions        1         5
5      Ana  fractions        1         6
6      Bob  fractions        0         1
7      Bob  fractions        0         2
8      Bob  fractions        1         3
9      Bob  fractions        1         4
10  Carlos  fractions        1         1
11  Carlos  fractions        0         2
12  Carlos  fractions        1         3


In [55]:
from pyBKT.models import Model

model = Model(seed=0)
model.fit(
    data=test,
    skills='fractions',
    multiprior='user_id',        # prior distinto para cada estudiante
    multilearn='item_id',        # learn rate distinto para cada estudiante
    forgets=True,                # activa olvido
    defaults={
        'skill_name':'skill_name',
        'correct':'correct',
        'order_id':'order_id',
        'multiprior':'user_id'   # mapeo de columna
    }
)

print(model.params().loc['fractions'])

ValueError: cannot specify more than 1 resource handling

In [51]:
from pyBKT.models import Roster

roster = Roster(['Ana','Bob','Carlos'], 'fractions', model=model)

for student in ['Ana','Bob','Carlos']:
    print(student, "prior =", roster.get_correct_prob('fractions', student))

Ana prior = 0.3893359722288841
Bob prior = 0.3893359722288841
Carlos prior = 0.3893359722288841


In [84]:
from pyBKT.models import Model

df_A = pd.DataFrame({
    'user_id':['Ana']*6,
    'skill_name':['fractions']*6,
    'problem_id':["A", "B", "C", "B", "C", "A"],
    'correct':[0,1,1,1,1,1],
    'order_id':range(1,7)
})

# ① ajuste inicial con lote A
model = Model(seed=0)
model.fit(data=df_A,
          multilearn='problem_id',
          defaults={'multilearn':'problem_id'})

print("=== Parámetros tras lote A ===")
print(model.params().loc['fractions'])

# ② aprendizaje incremental con lote B
model.partial_fit(data=df_A,  # ← usa el mismo objeto
                  multilearn='problem_id',
                  defaults={'multilearn':'problem_id'})

print("\n=== Parámetros tras lote A + B (partial_fit) ===")
print(model.params().loc['fractions'])


=== Parámetros tras lote A ===
                  value
param   class          
prior   default 0.00165
learns  A       0.99970
        B       0.99970
        C       0.99970
guesses default 0.00030
slips   default 0.00033
forgets A       0.00000
        B       0.00000
        C       0.00000

=== Parámetros tras lote A + B (partial_fit) ===
                  value
param   class          
prior   default 0.00000
learns  A       1.00000
        B       1.00000
        C       1.00000
guesses default 0.00000
slips   default 0.00000
forgets A       0.00000
        B       0.00000
        C       0.00000


In [83]:
print(model.fit_model['fractions']['resource_names'])


{'Default': 1, "['B'] ['A']": 2, "['C'] ['B']": 3, "['B'] ['C']": 4, "['A'] ['C']": 5}


In [88]:
import pandas as pd
from pyBKT.models import Model

# ---------- 1) modelo entrenado con multipair ----------
model = Model(seed=0)
model.fit(data=df_A, skills='fractions',
          multipair='problem_id',
          defaults={'multipair':'problem_id'})

# ---------- 2) construir DataFrame transición → learn / forget ----------
res   = model.fit_model['fractions']['resource_names']   # lista de pares
learn = model.fit_model['fractions']['learns']
forget= model.fit_model['fractions']['forgets']

res_names   = model.fit_model['fractions']['resource_names']
learn_rates = model.fit_model['fractions']['learns']
forget_rates= model.fit_model['fractions']['forgets']
print(res_names)
pairs_df = pd.DataFrame({
    'prev'  : [p[0] for p in res_names],       # tal cual, sin ordenar
    'curr'  : [p[1] for p in res_names],
    'learn' : learn_rates,
    'forget': forget_rates
})

# tablas ordenadas
table_learn  = pairs_df.sort_values('learn',  ascending=False)
table_forget = pairs_df.sort_values('forget', ascending=False)
# ──▶ muestra en consola o guarda a CSV
print("▲ Transiciones ordenadas por learn (mayor es mejor)\n")
print(table_learn)

print("\n▼ Transiciones ordenadas por forget (mayor es peor)\n")
print(table_forget)


{'Default': 1, "['B'] ['A']": 2, "['C'] ['B']": 3, "['B'] ['C']": 4, "['A'] ['C']": 5}
▲ Transiciones ordenadas por learn (mayor es mejor)

  prev curr   learn  forget
4    [    ' 1.00000 0.00000
0    D    e 0.99628 0.00000
1    [    ' 0.99628 0.00000
2    [    ' 0.99628 0.00000
3    [    ' 0.99628 0.00000

▼ Transiciones ordenadas por forget (mayor es peor)

  prev curr   learn  forget
0    D    e 0.99628 0.00000
1    [    ' 0.99628 0.00000
2    [    ' 0.99628 0.00000
3    [    ' 0.99628 0.00000
4    [    ' 1.00000 0.00000


In [101]:
import pandas as pd
from pyBKT.models import Model

# ─────────────────────────────────────────
# 1) Dataset de juguete con 3 ítems A, B, C
# ─────────────────────────────────────────
df = pd.DataFrame({
    'user_id'   : ['Ana','Ana','Ana','Ana',  'Bob','Bob','Bob','Bob'],
    'skill_name': ['fractions']*8,
    'item_id'   : ['A','B','C','A',          'A','C','B','C'],
    'correct'   : [0,1,1,1,                  0,0,1,1]
})
df['order_id'] = df.groupby(['user_id','skill_name']).cumcount() + 1

# ─────────────────────────────────────────
# 2) Entrenamos BKT con multilearn (+ olvido)
# ─────────────────────────────────────────
model = Model(seed=0)
model.fit(
    data=df,
    skills='fractions',
    multilearn='item_id',      # ▶︎ learn / forget por recurso
    forgets=True,              # ▶︎ permite estimar forgets por recurso
    defaults={
        'skill_name':'skill_name',
        'correct':'correct',
        'order_id':'order_id',
        'multilearn':'item_id'
    }
)

# ─────────────────────────────────────────
# 3) Extraemos parámetros y generamos tablas
# ─────────────────────────────────────────
res_names   = model.fit_model['fractions']['resource_names']   # ['A','B','C']
learn_rates = model.fit_model['fractions']['learns']           # np.array[3]
forget_rates= model.fit_model['fractions']['forgets']          # np.array[3]
# Invertir diccionario res_names
res_names = {item[1]: item[0] for item in res_names.items()}
# Get items of res_names by key
res_names = [res_names[item] for item in res_names]
print(res_names)
print(learn_rates)
print(forget_rates)
params_df = pd.DataFrame({
    'item'  : res_names,
    'learn' : learn_rates,
    'forget': forget_rates
})

# Tabla ordenada por learn (↑ mejor enseña)
learn_table  = params_df.sort_values('learn',  ascending=False).reset_index(drop=True)

# Tabla ordenada por forget (↑ más riesgo de olvido)
forget_table = params_df.sort_values('forget', ascending=False).reset_index(drop=True)

print("\n▲ Ítems ordenados por learn\n")
print(learn_table)

print("\n▼ Ítems ordenados por forget\n")
print(forget_table)


['A', 'B', 'C']
[0.53202208 0.99129373 0.99397655]
[6.79949558e-03 6.91230259e-05 8.81212374e-05]

▲ Ítems ordenados por learn

  item   learn  forget
0    C 0.99398 0.00009
1    B 0.99129 0.00007
2    A 0.53202 0.00680

▼ Ítems ordenados por forget

  item   learn  forget
0    A 0.53202 0.00680
1    C 0.99398 0.00009
2    B 0.99129 0.00007
